In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver # módulo que contiene implementaciones de controladores de navegadores web
from webdriver_manager.chrome import ChromeDriverManager # controlador de Chrome  
from selenium.webdriver.support import expected_conditions as EC # método para escribir códigos que esperan hasta que ciertas condiciones sean cumplidas
from selenium.webdriver.support.ui import WebDriverWait # método para escribir códigos que utilizan esperas implícitas o explícitas
from selenium.webdriver.common.by import By # método para localizar elementos por sus atributos HTML
from selenium.webdriver import ActionChains # módulo para implementar interacciones con el navegador from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from moneyed import Money
from pandas import DataFrame 
import pandas as pd


In [3]:
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
#d = webdriver.Chrome('your-chrome-driver-path',chrome_options=chrome_options)

In [4]:
# Inicializa la página web en el navegador Chrome
driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=chrome_options)
url = 'https://www.despegar.com.co/'
driver.get(url) # aca le pido al webdriver que me traiga la pagina que le solicite en url

[WDM] - Current google-chrome version is 97.0.4692
[WDM] - Get LATEST driver version for 97.0.4692
[WDM] - Driver [/home/genesis/.wdm/drivers/chromedriver/linux64/97.0.4692.71/chromedriver] found in cache


In [5]:
elemento_oferta = '.shifu-3-header .header-corners-container .header-product-item .shifu-3-button-circle.OFFERS'
delay = 10
try:
    element_present = EC.presence_of_element_located((By.CSS_SELECTOR, elemento_oferta))
    WebDriverWait(driver, delay).until(element_present)
except TimeoutError:
    print('Timed out waiting for page to load')
print(driver.current_url)


https://www.despegar.com.co/


In [6]:
element1 = driver.find_element_by_css_selector(elemento_oferta)
ActionChains(driver).click(element1).perform()
print(driver.current_url)

https://www.despegar.com.co/ofertas-de-viajes


In [7]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()


In [8]:
etiqueta_promociones = soup.select('slider-item')  # selecciona todos los elementos que tienen la clase slider-item

In [9]:
def chequear_nulo(etiqueta):
    if etiqueta is not None:
        return etiqueta.text
  

def chequear_imagen(etiqueta):
    if etiqueta is not None:
        return etiqueta['src']

for etiqueta in etiqueta_promociones:
    titulo = etiqueta.select_one('.offer-card-title.small-title ')
    if titulo == None:
        continue
    imagen = etiqueta.select_one('.offer-card-image .offer-card-image-main')
    categoria = etiqueta.select_one('.offer-card-product')
    precio_oferta = etiqueta.select_one('.offer-card-pricebox-price-amount ')
    precio_normal = etiqueta.select_one('.offer-card-pricebox-price .offer-card-pricebox-price-old')
    # print(chequear_nulo(titulo),chequear_imagen(imagen),chequear_nulo(categoria),chequear_nulo(precio_oferta),chequear_nulo(precio_normal), end='\n')

   

In [10]:


# crear una lista de diccionarios
promociones_lista = []
promociones = {}
for etiqueta in etiqueta_promociones:
    try:
        promociones['categoria'] = chequear_nulo(etiqueta.select_one('.offer-card-product'))
        promociones['titulo'] = chequear_nulo(etiqueta.select_one('.offer-card-title.small-title'))
        promociones['precio_oferta'] = chequear_nulo(etiqueta.select_one('.offer-card-pricebox-price-amount'))
        promociones['precio_normal'] = chequear_nulo(etiqueta.select_one('.offer-card-pricebox-price .offer-card-pricebox-price-old'))
        promociones['imagen'] = chequear_imagen(etiqueta.select_one('.offer-card-image .offer-card-image-main'))
    except:
        continue
    promociones_lista.append(promociones.copy())
print(promociones_lista)


[{'categoria': 'Paquete', 'titulo': 'Paquetes a Cartagena de Indias', 'precio_oferta': '322.740', 'precio_normal': ' $530.060 ', 'imagen': '//media.staticontent.com/media/pictures/1d51a0f8-99f8-4c68-9e92-f7ea744437ea/300x200'}, {'categoria': 'Paquete', 'titulo': 'Paquetes a Cali', 'precio_oferta': '364.000', 'precio_normal': ' $617.200 ', 'imagen': '//media.staticontent.com/media/pictures/a857f04c-3a2a-4201-93da-864454c20815/300x200'}, {'categoria': 'Paquete', 'titulo': 'Paquetes a San Andrés', 'precio_oferta': '960.498', 'precio_normal': ' $1.607.971 ', 'imagen': '//media.staticontent.com/media/pictures/517b832f-0640-4db1-922d-360494e1e3f8/300x200'}, {'categoria': 'Paquete', 'titulo': 'Paquetes a Cancún', 'precio_oferta': '2.195.791', 'precio_normal': ' $2.513.889 ', 'imagen': '//media.staticontent.com/media/pictures/5027b2a8-c69e-4892-8de6-6b2972a96712/300x200'}, {'categoria': 'Paquete', 'titulo': 'Paquetes a Ciudad de México', 'precio_oferta': '1.345.467', 'precio_normal': ' $1.509.

In [11]:
# crear un dataframe
df = pd.DataFrame(promociones_lista)
df.columns = ['categoria', 'titulo', 'precio_oferta', 'precio_normal', 'imagen']
df.to_csv('promociones.csv', index=False, encoding='utf-8-sig')
# print(df)